In [1]:
#imports
import pandas as pd
import os
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

### Analyse Genomad Taxonomy assignment

In [11]:
# iterate the sample files to read geNomad reports
# genomadReportPath = "/lustre/shared/wfsr-mcfa/projects/internships/luka/viral_metagenomics_pipeline/results/genomad/host_removed"
# total_viruses = []
# samples = []
# genomad_taxonomy = pd.DataFrame()
# for sample in os.listdir(genomadReportPath):
#     if sample.startswith("all"):
#         continue
#     ignore_dirs = ["none_checkv_filtered","genomad_and_blast_sample_viruse.csv"] #files to ignore
#     if sample in ignore_dirs:
#         continue
    
#     #geNomad report per sample
#     report_df = pd.read_csv(os.path.join(genomadReportPath, sample,"checkv_medium_to_high_contigs_summary",
#                                          "checkv_medium_to_high_contigs_virus_summary.tsv"), sep="\t")
    
#     #count how many contigs had virus score above 0.7 (classified as virus)
#     total_viruses.append(report_df.query("virus_score > 0.7").shape[0])
#     samples.append(sample)

#     # select the geNomad taxonomies per sample
#     report_df = report_df.query("virus_score > 0.7") # take only those that viruses
#     sample_genomad_taxonomy = pd.DataFrame({     
#     'sample': sample,
#     'contig_name': report_df['seq_name'],
#     "virus_score": report_df['virus_score'],
#     "taxonomy": report_df['taxonomy']
#     }) 

#     #concat the taxonomies
#     genomad_taxonomy = pd.concat([genomad_taxonomy, sample_genomad_taxonomy])


# # count how many of the good contigs were viral (> 0.7 virus score)
# viruses_df = pd.DataFrame({
#     "sample": samples,
#     "genomad viruses":total_viruses
#         })


# # write to file as supplimentary data   
# #genomad_taxonomy.to_csv("../supplimentary_data/geNomad_taxonomies_for_good_quality_viral_contigs.csv")

In [12]:

def process_genomad_reports(genomadReportPath):
    """
    Process geNomad reports for viral contigs and generate summary statistics and taxonomies.

    Parameters:
    genomadReportPath (str): The directory containing geNomad reports for each sample.

    Returns:
    pd.DataFrame: A DataFrame containing taxonomies for good quality viral contigs.
    pd.DataFrame: A DataFrame containing the count of good quality viral contigs for each sample.
    """

    total_viruses = []
    samples = []
    genomad_taxonomy = pd.DataFrame()

    # Iterate through sample directories in the genomadReportPath
    for sample in os.listdir(genomadReportPath):
        if sample.startswith("all"):
            continue
        ignore_dirs = ["none_checkv_filtered", "genomad_and_blast_sample_viruse.csv"]  # Files to ignore
        if sample in ignore_dirs:
            continue

        # Read geNomad report per sample
        report_df = pd.read_csv(os.path.join(genomadReportPath, sample, "checkv_medium_to_high_contigs_summary",
                                            "checkv_medium_to_high_contigs_virus_summary.tsv"), sep="\t")

        # Count how many contigs had a virus score above 0.7 (classified as virus)
        total_viruses.append(report_df.query("virus_score > 0.7").shape[0])
        samples.append(sample)

        # Select the geNomad taxonomies per sample
        report_df = report_df.query("virus_score > 0.7")  # Take only those classified as viruses
        sample_genomad_taxonomy = pd.DataFrame({
            'sample': sample,
            'contig_name': report_df['seq_name'],
            "virus_score": report_df['virus_score'],
            "taxonomy": report_df['taxonomy']
        })

        # Concatenate the taxonomies
        genomad_taxonomy = pd.concat([genomad_taxonomy, sample_genomad_taxonomy])

    # Create a DataFrame with the count of good quality viral contigs for each sample
    viruses_df = pd.DataFrame({
        "sample": samples,
        "genomad viruses": total_viruses
    })

    # Write taxonomies to a CSV file as supplementary data
    genomad_taxonomy.to_csv("../supplimentary_data/geNomad_taxonomies_for_good_quality_viral_contigs.csv")

    return genomad_taxonomy, viruses_df


In [16]:
genomadReportPath = "/lustre/shared/wfsr-mcfa/projects/internships/luka/viral_metagenomics_pipeline/results/genomad/host_removed"
genomad_taxonomy, viruses_df = process_genomad_reports(genomadReportPath)
genomad_taxonomy.head()

,sample,contig_name,virus_score,taxonomy
0,HCFKNDSX3_104762-001-002_47_GAAGGAAG-CCAGGATG_...,NODE_2_length_7715_cov_668.847654,0.9825,Viruses;Riboviria;Orthornavirae;Pisuviricota;P...
1,HCFKNDSX3_104762-001-002_47_GAAGGAAG-CCAGGATG_...,NODE_10_length_2480_cov_46.993200,0.9712,Viruses;Riboviria;Orthornavirae;Kitrinoviricot...
2,HCFKNDSX3_104762-001-002_47_GAAGGAAG-CCAGGATG_...,NODE_9_length_2491_cov_62.958968,0.9601,Viruses;Riboviria;Orthornavirae;Kitrinoviricot...
3,HCFKNDSX3_104762-001-002_47_GAAGGAAG-CCAGGATG_...,NODE_16_length_2179_cov_20.280702,0.9572,Viruses;Monodnaviria;Sangervirae;Phixviricota;...
4,HCFKNDSX3_104762-001-002_47_GAAGGAAG-CCAGGATG_...,NODE_3_length_4674_cov_144.778315,0.9398,Viruses;Riboviria


In [14]:
# examine how many contigs were classified at family level
num_family = 0
num_unclassified = 0

for taxon in genomad_taxonomy['taxonomy']:
    #count the number of unclassified
    if taxon == 'Unclassified':
        num_unclassified += 1
        #print(taxon)
        continue
    #count number classified at family level
    else:
        levels = len(taxon.split(';'))
        #print(levels)
        if levels > 4: #https://portal.nersc.gov/genomad/taxonomic_assignment.html#
            num_family +=1
 

    #break
print(f"{num_family} contigs were classified at family level and {num_unclassified} were unclassified")

460 contigs were classified at family level and 15 were unclassified


## Viruses Associated with human host

In [7]:
hostDbPath = "/lustre/shared/wfsr-mcfa/projects/internships/luka/viral_metagenomics_pipeline/scripts/virushostdb.tsv"

In [8]:
hostdb_viruses = pd.read_csv(hostDbPath, sep="\t", usecols=['virus tax id', "refseq id", "virus lineage", "host name", "DISEASE"])
hostdb_viruses.head(10)

,virus tax id,virus lineage,refseq id,DISEASE,host name
0,46350,Viruses; Monodnaviria; Shotokuvirae; Cossaviri...,NC_001729,NaN,Homo sapiens
1,57579,Viruses; Monodnaviria; Shotokuvirae; Cossaviri...,NC_001829,NaN,Macaca mulatta
2,57579,Viruses; Monodnaviria; Shotokuvirae; Cossaviri...,NC_001829,NaN,Homo sapiens
3,57579,Viruses; Monodnaviria; Shotokuvirae; Cossaviri...,NC_001829,NaN,Chlorocebus sabaeus
4,68558,Viruses; Monodnaviria; Shotokuvirae; Cossaviri...,AF028704,NaN,Homo sapiens
5,10804,Viruses; Monodnaviria; Shotokuvirae; Cossaviri...,NC_001401,NaN,Homo sapiens
6,10804,Viruses; Monodnaviria; Shotokuvirae; Cossaviri...,NC_001401,NaN,Mammalia
7,68742,Viruses; Monodnaviria; Shotokuvirae; Cossaviri...,AF028705,NaN,Homo sapiens
8,82300,Viruses; Monodnaviria; Shotokuvirae; Cossaviri...,NC_006152,NaN,Homo sapiens
9,335103,Viruses; Riboviria; Orthornavirae; Duplornavir...,"NC_007548, NC_007549, NC_007550, NC_007551, NC...",NaN,Homo sapiens


In [9]:
#select only viruses with human host
hostdb_viruses.query(" `host name` == 'Homo sapiens'", inplace= True)
hostdb_viruses.head()



,virus tax id,virus lineage,refseq id,DISEASE,host name
0,46350,Viruses; Monodnaviria; Shotokuvirae; Cossaviri...,NC_001729,NaN,Homo sapiens
2,57579,Viruses; Monodnaviria; Shotokuvirae; Cossaviri...,NC_001829,NaN,Homo sapiens
4,68558,Viruses; Monodnaviria; Shotokuvirae; Cossaviri...,AF028704,NaN,Homo sapiens
5,10804,Viruses; Monodnaviria; Shotokuvirae; Cossaviri...,NC_001401,NaN,Homo sapiens
7,68742,Viruses; Monodnaviria; Shotokuvirae; Cossaviri...,AF028705,NaN,Homo sapiens


In [10]:
blastout = "/lustre/shared/wfsr-mcfa/projects/internships/luka/viral_metagenomics_pipeline/results/checkv/host_removed/HCFKNDSX3_104762-001-002_44_TTAATCAG-CAGATTGG_L004/checkv_medium_to_high_blast_test.txt"
detectedViruses = pd.read_csv(blastout,    sep="\t", header=None)
detectedViruses.rename( columns={0:'contig name', 1:'taxid', 2: "sacc", 3: "sscinames", 4:"sblastnames", 5:"evalue", 6:"bitscore", 7:"pident"}, inplace=True)

# keep only one hit per contig (query) based on eval, bit score and  percent identity
detectedViruses.sort_values(["contig name", "pident", "evalue", "bitscore"], ascending= [True, False, True, False], inplace=True)
detectedViruses.drop_duplicates(subset=['contig name'], keep='first', inplace=True)
detectedViruses.sort_index(inplace=True)
detectedViruses.head(20) 

,contig name,taxid,sacc,sscinames,sblastnames,evalue,bitscore,pident
1,NODE_1_length_10532_cov_285.994810,29159,XM_011415281,Crassostrea gigas,bivalves,1.500000e+00,52.8,100.000
11,NODE_2_length_7700_cov_406.999868,11978,M86379,Feline calicivirus,viruses,8.200000e-07,73.1,100.000
28,NODE_4_length_4017_cov_182.179177,1940570,MW348569,Parvoviridae sp.,viruses,1.600000e-155,566.0,74.943
29,NODE_5_length_3448_cov_97.686239,246410,XM_001240667,Coccidioides immitis RS,ascomycete fungi,4.800000e-01,52.8,100.000
32,NODE_7_length_3159_cov_29.293206,186617,JX904654,uncultured marine virus,viruses,1.980000e-14,97.1,84.694
34,NODE_13_length_2673_cov_133.439120,29159,LR761639,Crassostrea gigas,bivalves,1.000000e-01,54.7,100.000
35,NODE_38_length_1701_cov_29.395807,1954248,MT208579,Circoviridae sp.,viruses,3.860000e-04,62.1,87.037
38,NODE_52_length_1503_cov_82.534884,2202954,MH618202,Circular genetic element sp.,viruses,1.110000e-68,276.0,83.117
42,NODE_61_length_1468_cov_163.879195,988011,OX411766,Orthosia incerta,moths,2.000000e-01,52.8,96.774
45,NODE_100_length_1306_cov_35.586938,1954248,MH617385,Circoviridae sp.,viruses,4.900000e-02,54.7,100.000


In [11]:
#select only viruses with human host
hostdb_viruses.query(" `host name` == 'Homo sapiens'", inplace= True)
human_viruses_taxids = [x for x in hostdb_viruses['virus tax id'] ]
human_viruses_refseq1ds = [x for x in hostdb_viruses["refseq id"] ] # MY BLAST DOESNT have refseqid; currently using subject accession number


 # select human viruses
human_viruses_per_bin = detectedViruses.query("taxid.isin(@human_viruses_taxids) | \
                                                sacc.isin(@human_viruses_refseq1ds)") #find common taxid or common refseq id. THERE DIFF BTWN refseq & ACCESSion NUM
        

human_viruses_per_bin

,contig name,taxid,sacc,sscinames,sblastnames,evalue,bitscore,pident


## Create a report of human viruses detected in our dataset

In [48]:
#load host db viruses
hostDbPath = "/lustre/shared/wfsr-mcfa/projects/internships/luka/viral_metagenomics_pipeline/scripts/virushostdb.tsv"
hostdb_viruses = pd.read_csv(hostDbPath, sep="\t", usecols=['virus tax id', "refseq id", "virus lineage", "host name", "DISEASE"])

#select only viruses with human host
hostdb_viruses.query(" `host name` == 'Homo sapiens'", inplace= True)
human_viruses_taxids = [x for x in hostdb_viruses['virus tax id'] ]
human_viruses_refseq1ds = [x for x in hostdb_viruses["refseq id"] ] # MY BLAST DOESNT have refseqid; currently using subject accession number

#Initiate empty dataframe to be filled with human viruses detected in our dataset and also one with all detected viruses
all_detected_human_viruses = pd.DataFrame(columns=[ 'contig name', 'taxid', "sacc", "sscinames", "sblastnames", "evalue", "bitscore","pident", "sample"])
all_detected_viruses = pd.DataFrame(columns=[ 'contig name', 'taxid', "sacc", "sscinames", "sblastnames", "evalue", "bitscore","pident", "sample"])


detectedVirusesPath = "/lustre/shared/wfsr-mcfa/projects/internships/luka/viral_metagenomics_pipeline/results/blast/host_removed"
samples = []
num_viruses = []
non_viral = []
for file in os.listdir(detectedVirusesPath):
    if file.endswith(".txt"):   
        continue     
    file_size = os.path.getsize(os.path.join(f'{detectedVirusesPath}',f'{file}', "blastn_search.tsv")) #some blast files are empty if there are no hits
    
    if  file_size == 0:
        continue
    
    
    detectedViruses = pd.read_csv(os.path.join(f'{detectedVirusesPath}',f'{file}', "blastn_search.tsv"), sep="\t", header=None)
    detectedViruses.rename( columns={0:'contig name', 1:'taxid', 2: "sacc", 3: "sscinames", 4:"sblastnames", 5:"evalue", 6:"bitscore", 7:"pident"}, inplace=True)
        
    # keep only one hit per contig (query) based on eval, bit score and  percent identity: this could lead to missing other closely similar sequences
    detectedViruses.sort_values(["contig name", "bitscore","pident", "evalue"], ascending= [True, False, False, True], inplace=True)
    detectedViruses.drop_duplicates(subset=['contig name'], keep='first', inplace=True)
    detectedViruses.sort_index(inplace=True)
        
    #add a column with sample info
    detectedViruses["sample"] = file
    #detectedViruses = detectedViruses.query("bitscore > 100")
    #merge all viruses
    all_detected_viruses = pd.concat([all_detected_viruses, detectedViruses])

    # select human viruses
    human_viruses_per_sample = detectedViruses.query("taxid.isin(@human_viruses_taxids) | \
                                                    sacc.isin(@human_viruses_refseq1ds)") #find common taxid or common refseq id. THERE DIFF BTWN refseq & ACCESSion NUM
    
    #check how many contigs were labeled as viruses by blast for comparison genomad
    viruses_only = detectedViruses.query("sblastnames == 'viruses'").shape[0]
    num_viruses.append(viruses_only)
    
    
    #check how many were not viruses
    not_viruses = detectedViruses.query("sblastnames != 'viruses'").shape[0]
    non_viral.append(not_viruses)
    
    samples.append(file)
    #print(file, detectedViruses.shape)
    
    #merge with human viruses for all bins
    all_detected_human_viruses = pd.concat([all_detected_human_viruses, human_viruses_per_sample])

#creat a df tracking the number of viruses per sample
blast_num_viruses = pd.DataFrame({
    "sample": samples,
    "blast viruses": num_viruses,
    "blast_non_viral": non_viral
    })

#merge blast viruses with genomad viruses
genomad_blast_viruses = viruses_df.merge(blast_num_viruses, on="sample")
genomad_blast_viruses.to_csv("../supplimentary_data/genomad_and_blast_sample_viruses.csv")
# write out to file 
all_detected_human_viruses.to_csv("../supplimentary_data/good_quality_blast_host_removed_detected_human_viruses.txt", sep='\t', columns=all_detected_human_viruses.columns)
all_detected_viruses.to_csv("../supplimentary_data/good_quality_blast_host_removed_all_detected_viruses.txt", sep='\t', columns=all_detected_human_viruses.columns)


In [49]:
blast_num_viruses.head(20)

,sample,blast viruses,blast_non_viral
0,HCFKNDSX3_104762-001-002_47_GAAGGAAG-CCAGGATG_...,3,1
1,HCFKNDSX3_104762-001-002_64_AACGCATT-CAACCTGC_...,8,2
2,HCFKNDSX3_104762-001-002_57_ATTATCAA-GAGAGTCG_...,4,2
3,HCFKNDSX3_104762-001-002_44_TTAATCAG-CAGATTGG_...,3,3
4,HCFKNDSX3_104762-001-002_53_TCTGCAAG-AAGGTGAA_...,18,1
5,HCFKNDSX3_104762-001-002_09_CATGATCG-GGAGAGTA_...,23,5
6,H7NKYDSX3_104762-001-003_56_TTAATCAG-CAGATTGG_...,19,4
7,HCFKNDSX3_104762-001-002_35_AGGTGCGA-GCCAGAAG_...,18,4
8,HCFKNDSX3_104762-001-002_27_CTGTGGCG-GGCTAGTG_...,4,0
9,H7NKYDSX3_104762-001-003_31_TCGCCTTG-GAATCAGC_...,16,5


In [42]:
#all_detected_human_viruses.drop(columns=['subject_id'], inplace=True) #this col is not helpful i drop it
all_detected_human_viruses.head()

,contig name,taxid,sacc,sscinames,sblastnames,evalue,bitscore,pident,sample
22,NODE_33_length_1738_cov_11.656735,743300,NC_038392,Human stool-associated circular virus NG13,viruses,0.000030,65.8,100.0,HCFKNDSX3_104762-001-002_57_ATTATCAA-GAGAGTCG_...
69,NODE_164_length_1860_cov_96.292556,743300,NC_038392,Human stool-associated circular virus NG13,viruses,0.000033,65.8,100.0,H7NKYDSX3_104762-001-003_31_TCGCCTTG-GAATCAGC_...


In [14]:
all_detected_viruses.shape

(279, 9)

### High confidence viruses

In [15]:
highconf_viruses = all_detected_viruses.query("bitscore > 100")
highconf_viruses.shape


(204, 9)

In [20]:
highconf_viruses.head()

,contig name,taxid,sacc,sscinames,sblastnames,evalue,bitscore,pident,sample
8,NODE_2_length_7715_cov_668.847654,11978,M86379,Feline calicivirus,viruses,0.000000e+00,14022.0,99.622,HCFKNDSX3_104762-001-002_47_GAAGGAAG-CCAGGATG_...
26,NODE_16_length_2179_cov_20.280702,2202644,MH572379,Microviridae sp.,viruses,0.000000e+00,2715.0,98.882,HCFKNDSX3_104762-001-002_47_GAAGGAAG-CCAGGATG_...
32,NODE_64_length_1318_cov_46.818640,186617,JX904221,uncultured marine virus,viruses,1.680000e-46,202.0,73.623,HCFKNDSX3_104762-001-002_47_GAAGGAAG-CCAGGATG_...
0,NODE_2_length_7705_cov_1225.754157,11978,M86379,Feline calicivirus,viruses,0.000000e+00,14020.0,99.622,HCFKNDSX3_104762-001-002_64_AACGCATT-CAACCTGC_...
18,NODE_11_length_4100_cov_167.561540,2202644,MH572286,Microviridae sp.,viruses,3.440000e-172,621.0,80.669,HCFKNDSX3_104762-001-002_64_AACGCATT-CAACCTGC_...


In [19]:
highconf_viruses.shape

(204, 9)

In [21]:
genomad_blast_viruses

,sample,genomad viruses,blast viruses
0,HCFKNDSX3_104762-001-002_47_GAAGGAAG-CCAGGATG_...,8,3
1,HCFKNDSX3_104762-001-002_64_AACGCATT-CAACCTGC_...,29,8
2,HCFKNDSX3_104762-001-002_57_ATTATCAA-GAGAGTCG_...,10,4
3,HCFKNDSX3_104762-001-002_44_TTAATCAG-CAGATTGG_...,15,3
4,HCFKNDSX3_104762-001-002_53_TCTGCAAG-AAGGTGAA_...,33,18
5,HCFKNDSX3_104762-001-002_09_CATGATCG-GGAGAGTA_...,53,23
6,H7NKYDSX3_104762-001-003_56_TTAATCAG-CAGATTGG_...,62,19
7,HCFKNDSX3_104762-001-002_35_AGGTGCGA-GCCAGAAG_...,37,18
8,HCFKNDSX3_104762-001-002_27_CTGTGGCG-GGCTAGTG_...,13,4
9,H7NKYDSX3_104762-001-003_31_TCGCCTTG-GAATCAGC_...,48,16


### Sequences labled as non-viral by BLAST

In [17]:
blast_non_viruses = highconf_viruses.query("sblastnames != 'viruses'")
blast_non_viruses.shape

(0, 9)

In [18]:
blast_non_viruses.head(40)

,contig name,taxid,sacc,sscinames,sblastnames,evalue,bitscore,pident,sample
